In [ ]:
import re

import matplotlib.pyplot as plt
from commonroad.common.util import Interval
from commonroad.visualization.mp_renderer import MPRenderer

from OpenSCENARIO2CR.AbsRel import AbsRel
from OpenSCENARIO2CR.EsminiWrapper.StoryBoardElement import EStoryBoardElementType
from OpenSCENARIO2CR.Osc2CrConverter import Osc2CrConverter

In [ ]:
# Setup Converter
converter = Osc2CrConverter(
    delta_t=0.1,
    osc_file="path/to/OpenSCENARIOfile.xosc",
    grace_time=1.0,
    ignored_level=EStoryBoardElementType.ACT,
    log_to_console=False,
    goal_state_time_step=AbsRel(interval=Interval(-5, 5), is_absolute=False),
    goal_state_position_length=50,
    goal_state_position_width=10,
    ego_filter=re.compile(r".*[Ee]go.*"),
    keep_ego_vehicle=False
)

scenario, planning_problem_set, statistics = converter.run_conversion()

In [ ]:
# plot every second time_stamp of the scenario
for i in range(0, max([obst.prediction.final_time_step for obst in scenario.dynamic_obstacles]) + 1, 2):
    plt.figure(figsize=(5, 5))

    plot_limit = 100 # If non-null the renderer follows the ego vehicle
    if plot_limit is None:
        rnd = MPRenderer()
    else:
        rnd = MPRenderer(plot_limits=[-plot_limit, plot_limit, -plot_limit, plot_limit],
                         draw_params={"focus_obstacle_id": scenario.obstacles[0].obstacle_id})
    # plot the scenario at different time step
    scenario.draw(rnd, draw_params={'time_begin': i})
    # plot the planning problem set
    planning_problem_set.draw(rnd)
    rnd.render()